In [1]:
import pandas as pd
import json

# The provided JSON-like string
data_str = '''
{
  "Name of resume holder": "Md Abdullah Al Hasib",
  "email": "alhasib.iu.cse@gmail.com",
  "is_perfect": false,
  "is_okay": true,
  "Matching Score in percentage": "75%",
  "strong zone": [
    "Hasib possesses hands-on experience in developing, testing, and deploying ML models, aligning with the job's core requirement.",
    "His proficiency in Python, experience with libraries like Pandas, Scikit-learn, and frameworks like TensorFlow directly match the job's technical needs.",
    "Hasib's experience with cloud platforms (AWS), Git, and Docker demonstrates his ability to work in the required development environment.",
    "His projects on License Plate Detection, Sentiment Analysis, and Forest Cover Type Prediction showcase practical ML application skills.",
    "Hasib's contributions to open-source projects highlight his commitment to the field and ability to collaborate on technical projects.",
    "His strong understanding of ML algorithms, problem-solving skills, and experience deploying models in production environments are highly valuable."
  ],
  "Lack of Knowledge": [
    "While the resume mentions experience with AWS, it lacks specifics about the extent of his cloud computing expertise, which is crucial for the role.",
    "The resume doesn't explicitly state his experience with SQL databases like Snowflake, PostgreSQL, or MySQL, which is a key requirement. ",
    "Although Hasib has experience with large datasets, his exposure to distributed computing needs to be clearer and might require further assessment.",
    "While his educational background in Computer Science is relevant, he should consider highlighting coursework or projects related to statistical modeling.",
    "The resume could benefit from quantifiable achievements in previous roles, demonstrating the impact of his ML contributions."
  ]
}
'''

# Convert string into Python dictionary
data_dict = json.loads(data_str)

# Convert dictionary to DataFrame
df = pd.DataFrame({
    "Name of resume holder": [data_dict["Name of resume holder"]],
    "email": [data_dict["email"]],
    "is_perfect": [data_dict["is_perfect"]],
    "is_okay": [data_dict["is_okay"]],
    "Matching Score in percentage": [data_dict["Matching Score in percentage"]],
    "strong zone": ["\n".join(data_dict["strong zone"])],  # Join list elements with newline for display
    "Lack of Knowledge": ["\n".join(data_dict["Lack of Knowledge"])]  # Join list elements with newline for display
})

# Display the DataFrame
print(df)


  Name of resume holder                     email  is_perfect  is_okay  \
0  Md Abdullah Al Hasib  alhasib.iu.cse@gmail.com       False     True   

  Matching Score in percentage  \
0                          75%   

                                         strong zone  \
0  Hasib possesses hands-on experience in develop...   

                                   Lack of Knowledge  
0  While the resume mentions experience with AWS,...  


In [9]:
import os
# import streamlit as st
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
import PyPDF2
from langchain_core.messages import AIMessage
import fitz
#pip install pymupdf
# import streamlit as st
import pandas as pd

load_dotenv()

api_key = os.getenv("GOOGLE_GEN_API")
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", api_key=api_key)

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You will compare the Resume and Job Description and provide the Output in a JSON format"),
    ("human", "You will take the resume and Job description: \n \
     Resume: {resume} \n \
     Job Description: {Job_description} \n \
     Based on the Job description, does the resume is perfect for the JOB and What's the percentage of Matching the resume against the JOB description. \
     The JSON Output Key will be Name of resume holder, email, is_perfect,is_okay, Matching Score in percentage, strong zone, Lack of Knowledge. Please provide the answer for strong zon and lack of knowledge in string, not list")
])

# Function to extract text from PDF
def load_pdf(pdf_file):
    pdf_document = fitz.open(pdf_file)  # Open the PDF
    pdf_text_with_links = ""

    # Iterate through each page
    for page_num in range(pdf_document.page_count):
        page = pdf_document.load_page(page_num)  # Load individual page
        pdf_text_with_links += page.get_text("text")  # Extract text

        # Extract links from the page
        links = page.get_links()
        for link in links:
            if 'uri' in link:  # Check if it's a URI (URL)
                pdf_text_with_links += f"\n(Link: {link['uri']})"
    
    return pdf_text_with_links
def parser(aimessage:AIMessage)->str:
    return aimessage.content


# Function to generate quiz using LangChain
def resume_checker(resume, job_description):
    Chain = prompt_template | llm | parser
    result = Chain.invoke({"resume":resume, "Job_description":job_description})
    return result

resume = load_pdf(r"C:\Users\abdullah\projects\Langchain\Generative-AI\LangChain\RAG\data\My_CV_2.pdf")
# print(resume)

Job_description = """ Job Responsibilities

 Leverage cutting-edge AI/ML methods to develop AI/ML-powered solutions that align with business objectives.
 Prepare dataset/data-pipeline to feed into the machine learning models.
 Optimize ML applications to improve performance, speed, and scalability
 Work with internal stakeholders, analysts, engineers to explore and understand business data.
 Conduct experiments and test different technical approaches
 Collaborate with the data engineering team to ensure clean records
 Implement machine learning model lifecycle management framework.
 Build/maintain model performance tracking tools, to monitor model performance, identify drivers and provide recommendations for optimization
 Author technical documentation and reports to communicate process and results

Knowledge And Experience

Minimum qualifications:

 1 to 4 years of hands-on experience in machine learning models, applications, and pipelines. 
 Experience in developing, testing, debugging, maintaining, or launching ML products, as well as software design and architecture. 
 Proficiency in one or more programming languages (preferably Python). 
 Working knowledge of one or more SQL languages (preferably Snowflake, PostgreSQL, MySQL). 
 Knowledge of common machine learning and statistical packages, frameworks, and concepts. 
 Experience/knowledge of working with large data sets, distributed computing, and cloud computing platforms. 
 Experience using CI/CD tools and version control systems (e.g., Git). 
 Excellent written and verbal communication skills in English. 

Preferred qualifications:

 Strong understanding of machine learning algorithms and their practical implementation. 
 Excellent problem-solving skills and the ability to translate business requirements into technical solutions. 
 Experience deploying machine learning models in production environments. 
 Excellent analytical and problem-solving skills with keen attention to detail. 
 A team-oriented mindset, believing that AI/ML is a collaborative effort. 

Education

Bachelor’s degree in Computer Science, Mathematics, Statistics, Physics, or a related field with project experience in machine learning and statistical modeling.

Competencies

Driving Continuous Improvement

Driving for Results

Driving Projects to Completion

Interacting with People at Different Levels

Using Computers and Technology
"""

# print(llm.invoke("How can i create an application for resume checking for any JOB descrition"))
# checker = resume_checker(resume=resume, job_description=Job_description)
# print(checker)

# st.title("Resume Screening Project")

# File uploader
# uploaded_file = st.file_uploader("Upload a PDF file ", type="pdf")
# # print(uploaded_file)
# st.write(load_pdf(uploaded_file))

# pdf_path = st.text_input("Enter the path of the Resume")
# resume = (load_pdf(resume))
# st.write(resume)

# Create a large text area for user input
# Job_description = st.text_area("Enter Job Description:", height=300)
# st.write(Job_description)

checker = resume_checker(resume=resume, job_description=Job_description)

print(type(checker))

# st.write(checker)
import json
# Convert to DataFrame
data = json.loads(str(checker))

df = pd.DataFrame({
    "Name of resume holder": [data["Name of resume holder"]],
    "email": [data["email"]],
    "is_perfect": [data["is_perfect"]],
    "is_okay": [data["is_okay"]],
    "Matching Score in percentage": [data["Matching Score in percentage"]],
    "strong zone": ["\n".join(data["strong zone"])],
    "Lack of Knowledge": ["\n".join(data["Lack of Knowledge"])]
})

df.to_csv("resume screening.csv", index=False)

# Display the dataframe
# st.write(df)

<class 'str'>


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [10]:
print(checker)

```json
{
  "Name of resume holder": "Md Abdullah Al Hasib",
  "email": "alhasib.iu.cse@gmail.com",
  "is_perfect": false,
  "is_okay": true,
  "Matching Score in percentage": "75%",
  "strong zone": "Md Abdullah Al Hasib has a strong background in machine learning, deep learning, and computer vision as evidenced by his projects and experience. He is proficient in Python and familiar with various ML libraries and frameworks. His experience with cloud platforms and tools like Docker aligns well with the job requirements. Additionally, his contributions to open-source projects and leadership experience demonstrate his commitment to the field and his ability to work in a team. ",
  "Lack of Knowledge": "While he possesses some experience, explicitly stating his experience with SQL databases (Snowflake, PostgreSQL, MySQL) in the context of data analysis and machine learning would strengthen his application.  Further, elaborating on experience with CI/CD tools and version control systems li

In [11]:
print(str(checker))

```json
{
  "Name of resume holder": "Md Abdullah Al Hasib",
  "email": "alhasib.iu.cse@gmail.com",
  "is_perfect": false,
  "is_okay": true,
  "Matching Score in percentage": "75%",
  "strong zone": "Md Abdullah Al Hasib has a strong background in machine learning, deep learning, and computer vision as evidenced by his projects and experience. He is proficient in Python and familiar with various ML libraries and frameworks. His experience with cloud platforms and tools like Docker aligns well with the job requirements. Additionally, his contributions to open-source projects and leadership experience demonstrate his commitment to the field and his ability to work in a team. ",
  "Lack of Knowledge": "While he possesses some experience, explicitly stating his experience with SQL databases (Snowflake, PostgreSQL, MySQL) in the context of data analysis and machine learning would strengthen his application.  Further, elaborating on experience with CI/CD tools and version control systems li

In [21]:
# Replace ``` with """ and remove json
x = checker.replace('```json', "").replace('```', "")
x

'\n{\n  "Name of resume holder": "Md Abdullah Al Hasib",\n  "email": "alhasib.iu.cse@gmail.com",\n  "is_perfect": false,\n  "is_okay": true,\n  "Matching Score in percentage": "75%",\n  "strong zone": "Md Abdullah Al Hasib has a strong background in machine learning, deep learning, and computer vision as evidenced by his projects and experience. He is proficient in Python and familiar with various ML libraries and frameworks. His experience with cloud platforms and tools like Docker aligns well with the job requirements. Additionally, his contributions to open-source projects and leadership experience demonstrate his commitment to the field and his ability to work in a team. ",\n  "Lack of Knowledge": "While he possesses some experience, explicitly stating his experience with SQL databases (Snowflake, PostgreSQL, MySQL) in the context of data analysis and machine learning would strengthen his application.  Further, elaborating on experience with CI/CD tools and version control systems 

In [16]:
x

').replace(\'```\', \n{\n  "Name of resume holder": "Md Abdullah Al Hasib",\n  "email": "alhasib.iu.cse@gmail.com",\n  "is_perfect": false,\n  "is_okay": true,\n  "Matching Score in percentage": "75%",\n  "strong zone": "Md Abdullah Al Hasib has a strong background in machine learning, deep learning, and computer vision as evidenced by his projects and experience. He is proficient in Python and familiar with various ML libraries and frameworks. His experience with cloud platforms and tools like Docker aligns well with the job requirements. Additionally, his contributions to open-source projects and leadership experience demonstrate his commitment to the field and his ability to work in a team. ",\n  "Lack of Knowledge": "While he possesses some experience, explicitly stating his experience with SQL databases (Snowflake, PostgreSQL, MySQL) in the context of data analysis and machine learning would strengthen his application.  Further, elaborating on experience with CI/CD tools and versi

In [24]:
xx = json.loads(x)
data = xx

In [25]:
# Create a DataFrame from the dictionary
df = pd.DataFrame({
    "Name of resume holder": [data["Name of resume holder"]],
    "email": [data["email"]],
    "is_perfect": [data["is_perfect"]],
    "is_okay": [data["is_okay"]],
    "Matching Score in percentage": [data["Matching Score in percentage"]],
    "strong zone": [data["strong zone"]],
    "Lack of Knowledge": [data["Lack of Knowledge"]]
})

# Display the DataFrame
print(df)

  Name of resume holder                     email  is_perfect  is_okay  \
0  Md Abdullah Al Hasib  alhasib.iu.cse@gmail.com       False     True   

  Matching Score in percentage  \
0                          75%   

                                         strong zone  \
0  Md Abdullah Al Hasib has a strong background i...   

                                   Lack of Knowledge  
0  While he possesses some experience, explicitly...  


In [26]:
df.to_csv('resume_screening.csv', index=False)

In [27]:
import pandas as pd
import json

# Initialize a list to hold the data
data_list = []

# JSON string to add in each iteration
json_string = '''{
  "Name of resume holder": "Md Abdullah Al Hasib",
  "email": "alhasib.iu.cse@gmail.com",
  "is_perfect": false,
  "is_okay": true,
  "Matching Score in percentage": "75%",
  "strong zone": "Md Abdullah Al Hasib has a strong background in machine learning, deep learning, and computer vision as evidenced by his projects and experience. He is proficient in Python and familiar with various ML libraries and frameworks. His experience with cloud platforms and tools like Docker aligns well with the job requirements. Additionally, his contributions to open-source projects and leadership experience demonstrate his commitment to the field and his ability to work in a team.",
  "Lack of Knowledge": "While he possesses some experience, explicitly stating his experience with SQL databases (Snowflake, PostgreSQL, MySQL) in the context of data analysis and machine learning would strengthen his application. Further, elaborating on experience with CI/CD tools and version control systems like Git, and how they were used in deploying machine learning models would be beneficial."
}'''

# Loop to add the data multiple times
for i in range(4):
    # Parse the JSON string to a dictionary
    data_dict = json.loads(json_string)
    data_list.append(data_dict)

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data_list)

# Display the DataFrame
print(df)

# Optionally, save to CSV
df.to_csv("resume_data.csv", index=False)


  Name of resume holder                     email  is_perfect  is_okay  \
0  Md Abdullah Al Hasib  alhasib.iu.cse@gmail.com       False     True   
1  Md Abdullah Al Hasib  alhasib.iu.cse@gmail.com       False     True   
2  Md Abdullah Al Hasib  alhasib.iu.cse@gmail.com       False     True   
3  Md Abdullah Al Hasib  alhasib.iu.cse@gmail.com       False     True   

  Matching Score in percentage  \
0                          75%   
1                          75%   
2                          75%   
3                          75%   

                                         strong zone  \
0  Md Abdullah Al Hasib has a strong background i...   
1  Md Abdullah Al Hasib has a strong background i...   
2  Md Abdullah Al Hasib has a strong background i...   
3  Md Abdullah Al Hasib has a strong background i...   

                                   Lack of Knowledge  
0  While he possesses some experience, explicitly...  
1  While he possesses some experience, explicitly...  
2  While he 

In [ ]:
'''
Job_description = """ Job Responsibilities

 Leverage cutting-edge AI/ML methods to develop AI/ML-powered solutions that align with business objectives.
 Prepare dataset/data-pipeline to feed into the machine learning models.
 Optimize ML applications to improve performance, speed, and scalability
 Work with internal stakeholders, analysts, engineers to explore and understand business data.
 Conduct experiments and test different technical approaches
 Collaborate with the data engineering team to ensure clean records
 Implement machine learning model lifecycle management framework.
 Build/maintain model performance tracking tools, to monitor model performance, identify drivers and provide recommendations for optimization
 Author technical documentation and reports to communicate process and results

Knowledge And Experience

Minimum qualifications:

 1 to 4 years of hands-on experience in machine learning models, applications, and pipelines. 
 Experience in developing, testing, debugging, maintaining, or launching ML products, as well as software design and architecture. 
 Proficiency in one or more programming languages (preferably Python). 
 Working knowledge of one or more SQL languages (preferably Snowflake, PostgreSQL, MySQL). 
 Knowledge of common machine learning and statistical packages, frameworks, and concepts. 
 Experience/knowledge of working with large data sets, distributed computing, and cloud computing platforms. 
 Experience using CI/CD tools and version control systems (e.g., Git). 
 Excellent written and verbal communication skills in English. 

Preferred qualifications:

 Strong understanding of machine learning algorithms and their practical implementation. 
 Excellent problem-solving skills and the ability to translate business requirements into technical solutions. 
 Experience deploying machine learning models in production environments. 
 Excellent analytical and problem-solving skills with keen attention to detail. 
 A team-oriented mindset, believing that AI/ML is a collaborative effort. 

Education

Bachelor’s degree in Computer Science, Mathematics, Statistics, Physics, or a related field with project experience in machine learning and statistical modeling.

Competencies

Driving Continuous Improvement

Driving for Results

Driving Projects to Completion

Interacting with People at Different Levels

Using Computers and Technology 
"""
'''


#resume = load_pdf(r"C:\Users\abdullah\projects\Langchain\Generative-AI\LangChain\RAG\data\My_CV_2.pdf")
# print(resume)



In [2]:
# Import smtplib for the actual sending function
import smtplib

# Import the email modules we'll need
from email.mime.text import MIMEText

# Open a plain text file for reading.  For this example, assume that
# the text file contains only ASCII characters.

msg = MIMEText("Hi how are you?")

you ='alhasib.iu.cse@gmail.com'
me ='mdabdullahalhasib1111@gmail.com'
msg['Subject'] = 'The contents of'
msg['To'] = 'alhasib.iu.cse@gmail.com'
msg['From'] = 'mdabdullahalhasib1111@gmail.com'

# Send the message via our own SMTP server, but don't include the
# envelope header.
s = smtplib.SMTP('localhost')
s.sendmail(me, [you], msg.as_string())
s.quit()

ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

In [4]:
import smtplib
import pandas as pd
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# Step 1: Read the CSV file
csv_file_path = r'C:\Users\abdullah\projects\Langchain\Generative-AI\LangChain\RAG\resume_data.csv'  # Replace with your CSV file path
data = pd.read_csv(csv_file_path)

# Assuming the CSV has two columns: 'email' and 'message'
email_list = data['email'].tolist()
message_list = data['strong zone'].tolist()

# Step 2: Define the SMTP server details
smtp_server = 'smtp.gmail.com'  # For Gmail
smtp_port = 587
your_email = 'mdabdullahalhasib1111@gmail.com'  # Replace with your email
your_password = '5a6b7c8d'  # Replace with your email password

# Step 3: Establish a connection to the SMTP server
server = smtplib.SMTP(smtp_server, smtp_port)
server.starttls()  # Secure the connection
server.login(your_email, your_password)

# Step 4: Send an email to each recipient
for email, message in zip(email_list, message_list):
    msg = MIMEMultipart()
    msg['From'] = your_email
    msg['To'] = email
    msg['Subject'] = 'Your Subject Here'  # Customize your subject line
    
    # Attach the message body
    msg.attach(MIMEText(message, 'plain'))
    
    # Send the email
    server.sendmail(your_email, email, msg.as_string())

# Step 5: Close the SMTP server connection
server.quit()

print("Emails sent successfully!")


gaierror: [Errno 11001] getaddrinfo failed

In [1]:
import smtplib

# list of email_id to send the mail
li = ["alhasib.iu.cse@gmail.com", "mdabdullahalhasib1111@gmail.com"]

for dest in li:
    s = smtplib.SMTP('smtp.gmail.com', 587)
    s.starttls()
    s.login("mdabdullahalhasib2222@gmail.com", "5a6b7c8d")
    message = "Message_you_need_to_send"
    s.sendmail("mdabdullahalhasib2222@gmail.com", dest, message)
    s.quit()

SMTPAuthenticationError: (534, b'5.7.9 Application-specific password required. For more information, go to\n5.7.9  https://support.google.com/mail/?p=InvalidSecondFactor d9443c01a7336-20c8c21d953sm51119875ad.214 - gsmtp')

In [2]:
# Import smtplib for the actual sending function
import smtplib

# Import the email modules we'll need
from email.message import EmailMessage

# Open the plain text file whose name is in textfile for reading.

msg = EmailMessage()
msg.set_content("Hi")

# me == the sender's email address
# you == the recipient's email address
msg['Subject'] = f'The contents of'
msg['From'] = "alhasib.iu.cse@gmail.com"
msg['To'] = "mdabdullahalhasib1111@gmail.com"

# Send the message via our own SMTP server.
s = smtplib.SMTP('localhost')
s.send_message(msg)
s.quit()

ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

In [8]:
import yagmail
yag = yagmail.SMTP('alhasib.iu.cse@gmail.com', 'iyto heuk fbka mysg')
contents = ['This is the body, and here is just text, You can find an audio file attached.']
yag.send('alhasib.iu.cse@gmail.com', 'subject', contents)

{}

In [20]:
import pandas as pd

data = pd.read_csv(r"C:\Users\abdullah\projects\Langchain\Generative-AI\LangChain\RAG\resume_screening.csv")

print(data.head)

<bound method NDFrame.head of   Name of resume holder                     email  is_perfect  is_okay  \
0  Md Abdullah Al Hasib  alhasib.iu.cse@gmail.com       False     True   

  Matching Score in percentage  \
0                          75%   

                                         strong zone  \
0  Md Abdullah Al Hasib has a strong background i...   

                                   Lack of Knowledge  
0  While he possesses some experience, explicitly...  >


In [15]:
data.iloc[0]["Lack of Knowledge"]

'While he possesses some experience, explicitly stating his experience with SQL databases (Snowflake, PostgreSQL, MySQL) in the context of data analysis and machine learning would strengthen his application.  Further, elaborating on experience with CI/CD tools and version control systems like Git, and how they were used in deploying machine learning models would be beneficial.'

In [17]:
data.shape[0]

1

In [24]:


yag = yagmail.SMTP('alhasib.iu.cse@gmail.com', 'iyto heuk fbka mysg')
data = pd.read_csv(r"C:\Users\abdullah\projects\Langchain\Generative-AI\LangChain\RAG\resume_screening.csv")

for i in range(data.shape[0]):
    data = data.iloc[i]
    name = data['Name of resume holder']
    email = data['email']
    score = data['Matching Score in percentage']
    if "%" in score:
        score = int(score.replace("%",""))
    else:
        score = int(score)
    strong_zone = data['strong zone']
    weak_zone = data['Lack of Knowledge']

    # print(name, score, strong_zone, week_zone)
    if score < 70:
        message = f"Hello {name},\n\n \
        Thank you for your application on Machine Learning position in our company. But we can't consider you for the further process. We find some week zones given below that doesn't match with our company.\
        you can upgrade yourself & try later.\n\n \
            {weak_zone}\n\n \
        Best Regards,\n Md Abdullah Al Hasib"
    
    else:
        message = f"Hello {name},\n\n \
        Thank you for your application on Machine Learning position in our company. We  consider you for the further process. \n\n \
        Best Regards,\n \
        Md Abdullah Al Hasib"
    
    yag.send(email, 'Response to Your ML Engineer Application', message)
